In [1]:
import os
import shutil
import zipfile
from google.colab import drive

# 1. Mount Drive
drive.mount('/content/drive')

# 2. Define the target directory for the extracted dataset
DATASET_TARGET_DIR = "/content/drive/MyDrive/FinalProject/"
TRAIN_PATH = os.path.join(DATASET_TARGET_DIR, "train")
TEST_PATH = os.path.join(DATASET_TARGET_DIR, "test")
LABELS_CSV = os.path.join(DATASET_TARGET_DIR, "train_labels.csv")

ValueError: mount failed

In [2]:
import numpy as np
import pandas as pd
import pathlib
import random
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.applications import EfficientNetB0
import os

# --- ULTRA CONFIG ---
IMG_HEIGHT = 224  # Standard EfficientNet
IMG_WIDTH = 224
BATCH_SIZE = 16   # Reduced to fit in GPU RAM
SEED = 42

# Reproducibility
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

print(f"Config: EfficientNetB0 | {IMG_HEIGHT}x{IMG_WIDTH} | Batch {BATCH_SIZE}")

Config: EfficientNetB0 | 224x224 | Batch 16


In [3]:
# 1. Load Labels
labels_df = pd.read_csv(LABELS_CSV)
class_names = sorted(labels_df["Label"].unique().tolist())
num_classes = len(class_names)
label_to_index = {label: idx for idx, label in enumerate(class_names)}
id_to_label = dict(zip(labels_df["Id"].astype(str).str.zfill(5), labels_df["Label"]))

train_image_paths = sorted([str(p) for p in pathlib.Path(TRAIN_PATH).glob("*.png")])
train_labels = []
valid_paths = []

for path in train_image_paths:
    img_id = os.path.splitext(os.path.basename(path))[0]
    if img_id in id_to_label:
        train_labels.append(label_to_index[id_to_label[img_id]])
        valid_paths.append(path)

# 2. Load Functions
def load_img(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img, channels=3)

    # Feature Size (Original)
    orig_shape = tf.cast(tf.shape(img)[:2], tf.float32)
    size_feat = orig_shape / 256.0

    # Resize High-Res (EfficientNet handles 0-255 internally usually, but explicit resize needed)
    img = tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])
    return img, size_feat, label

def augment(img, size, label):
    # Data Augmentation (Train only)
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_brightness(img, max_delta=0.2)
    img = tf.image.random_contrast(img, lower=0.8, upper=1.2)
    img = tf.image.random_saturation(img, 0.8, 1.2)
    # Random Zoom/Crop technique
    zoom_size = int(IMG_HEIGHT * 1.1)
    img = tf.image.resize(img, [zoom_size, zoom_size])
    # Corrected random_crop usage
    img = tf.image.random_crop(img, size=[IMG_HEIGHT, IMG_WIDTH, 3])
    return {"img": img, "size": size}, label

def no_augment(img, size, label):
    return {"img": img, "size": size}, label

# 3. Create Datasets
ds_full = tf.data.Dataset.from_tensor_slices((valid_paths, train_labels))
ds_full = ds_full.shuffle(len(valid_paths), seed=SEED)

val_size = int(0.2 * len(valid_paths))
train_ds = ds_full.skip(val_size).map(load_img, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.map(augment, num_parallel_calls=tf.data.AUTOTUNE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_ds = ds_full.take(val_size).map(load_img, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.map(no_augment, num_parallel_calls=tf.data.AUTOTUNE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

print("Pipeline ready with Augmentation.")

Pipeline ready with Augmentation.


In [4]:
def build_model():
    img_input = layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3), name="img")
    size_input = layers.Input(shape=(2,), name="size")

    # EfficientNetB0 (Pre-trained)
    # Note: EfficientNet includes normalization layers, expecting [0, 255] input.
    base_model = EfficientNetB0(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
                                include_top=False,
                                weights='imagenet')
    base_model.trainable = False # Frozen start

    x = base_model(img_input)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.5)(x)

    # Fusion
    combined = layers.Concatenate()([x, size_input])
    combined = layers.Dense(512, activation="relu", kernel_regularizer=regularizers.l2(1e-4))(combined)
    combined = layers.Dropout(0.5)(combined)

    output = layers.Dense(num_classes, activation="softmax")(combined)

    model = models.Model(inputs=[img_input, size_input], outputs=output)
    return model

model = build_model()
model.compile(optimizer="adam",
              loss=tf.keras.losses.SparseCategoricalCrossentropy(), # Label smoothing complex with sparse, keeping standard
              metrics=["accuracy"])
model.summary()

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ img (InputLayer)    │ (None, 224, 224,  │          0 │ -                 │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ efficientnetb0      │ (None, 7, 7,      │  4,049,571 │ img[0][0]         │
│ (Functional)        │ 1280)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 1280)      │          0 │ efficientnetb0[0… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 1280)      │          0 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ size (InputLayer)   │ (None, 2)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 1282)      │          0 │ dropout[0][0],    │
│ (Concatenate)       │                   │            │ size[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 512)       │    656,896 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 512)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 7)         │      3,591 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,710,058 (17.97 MB)

 Trainable params: 660,487 (2.52 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [5]:
print("Phase 1: Warmup (Head only)...")
history_warm = model.fit(train_ds, validation_data=val_ds, epochs=4)

Phase 1: Warmup (Head only)...
Epoch 1/4
314/314 ━━━━━━━━━━━━━━━━━━━━ 245s 664ms/step - accuracy: 0.5206 - loss: 1.3464 - val_accuracy: 0.8287 - val_loss: 0.6121
Epoch 2/4
314/314 ━━━━━━━━━━━━━━━━━━━━ 30s 94ms/step - accuracy: 0.7384 - loss: 0.8100 - val_accuracy: 0.8773 - val_loss: 0.4891
Epoch 3/4
314/314 ━━━━━━━━━━━━━━━━━━━━ 29s 91ms/step - accuracy: 0.7829 - loss: 0.6803 - val_accuracy: 0.9060 - val_loss: 0.4100
Epoch 4/4
314/314 ━━━━━━━━━━━━━━━━━━━━ 28s 89ms/step - accuracy: 0.8031 - loss: 0.6445 - val_accuracy: 0.9116 - val_loss: 0.3954


In [6]:
print("Phase 2: Fine-Tuning (Full Model)...")

model.trainable = True

# Crucial Callbacks
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6)

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), # Very low LR
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

history_fine = model.fit(train_ds,
                         validation_data=val_ds,
                         epochs=20,
                         callbacks=[early_stop, reduce_lr])

Phase 2: Fine-Tuning (Full Model)...
Epoch 1/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 71s 161ms/step - accuracy: 0.8190 - loss: 0.6125 - val_accuracy: 0.9124 - val_loss: 0.3991 - learning_rate: 1.0000e-05
Epoch 2/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 28s 90ms/step - accuracy: 0.8316 - loss: 0.5892 - val_accuracy: 0.9163 - val_loss: 0.3831 - learning_rate: 1.0000e-05
Epoch 3/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 29s 91ms/step - accuracy: 0.8241 - loss: 0.6025 - val_accuracy: 0.9139 - val_loss: 0.3878 - learning_rate: 1.0000e-05
Epoch 4/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 28s 90ms/step - accuracy: 0.8307 - loss: 0.6034 - val_accuracy: 0.9243 - val_loss: 0.3692 - learning_rate: 1.0000e-05
Epoch 5/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 28s 89ms/step - accuracy: 0.8386 - loss: 0.5538 - val_accuracy: 0.9179 - val_loss: 0.3760 - learning_rate: 1.0000e-05
Epoch 6/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 41s 90ms/step - accuracy: 0.8284 - loss: 0.6015 - val_accuracy: 0.9275 - val_loss: 0.3566 - learning_rate: 1.0000e-05
Epoch 7/20
314

In [7]:
import tqdm
from google.colab import files

print("TTA Predictions (Test Time Augmentation)...")

test_paths = sorted([str(p) for p in pathlib.Path(TEST_PATH).glob("*.png")])
submission_ids = []
predictions = []

for path in tqdm.tqdm(test_paths):
    # Load raw
    img_raw = tf.io.read_file(path)
    img = tf.image.decode_png(img_raw, channels=3)

    # Feature
    orig_shape = tf.cast(tf.shape(img)[:2], tf.float32)
    size_feat = orig_shape / 256.0

    # Resize
    img = tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH]) # [0-255] float32

    # Create Batch: [Original, Flipped]
    img_flip = tf.image.flip_left_right(img)
    batch_imgs = tf.stack([img, img_flip])
    batch_sizes = tf.stack([size_feat, size_feat])

    # Predict
    preds = model.predict({"img": batch_imgs, "size": batch_sizes}, verbose=0)

    # Average probabilities
    avg_pred = np.mean(preds, axis=0)
    predictions.append(class_names[np.argmax(avg_pred)])
    submission_ids.append(os.path.splitext(os.path.basename(path))[0])

# Save
sub_df = pd.DataFrame({"Id": submission_ids, "Label": predictions})
sub_df.to_csv("submission_ultimate.csv", index=False)
print("submission_ultimate.csv generated.")
files.download("submission_ultimate.csv")

TTA Predictions (Test Time Augmentation)...


0it [00:00, ?it/s]

submission_ultimate.csv generated.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Ensure Dataset is Extracted

It appears the test dataset might not have been extracted yet, leading to an empty submission file. This step will check if the `train` and `test` directories exist, and if not, it will attempt to extract them from a `finalproject.zip` file located in your Google Drive.

In [8]:
import os
import shutil
import zipfile

# Define paths
zip_path = "/content/drive/MyDrive/finalproject.zip" # Adjust if your zip file is named differently or in another location

# Check if dataset directories exist
if not os.path.exists(TRAIN_PATH) or not os.path.exists(TEST_PATH):
    print("Dataset directories (train/test) not found. Attempting to extract from zip file...")
    if os.path.exists(zip_path):
        try:
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(DATASET_TARGET_DIR)
            print(f"Dataset extracted to {DATASET_TARGET_DIR}")
        except Exception as e:
            print(f"Error extracting zip file: {e}")
            print("Please ensure 'finalproject.zip' is correctly placed and not corrupted.")
    else:
        print(f"Error: Zip file not found at {zip_path}.")
        print("Please upload 'finalproject.zip' to your Google Drive or specify the correct path.")
else:
    print("Dataset directories already exist. Skipping extraction.")

# Verify that images are now present in TEST_PATH
if not os.listdir(TEST_PATH):
    print(f"Warning: {TEST_PATH} is still empty after extraction attempt. Please check the zip file content and path.")
else:
    print(f"Found {len(os.listdir(TEST_PATH))} files in {TEST_PATH}")

Dataset directories already exist. Skipping extraction.


After running the above cell to ensure the dataset is present, you will need to re-run the cells that load the image paths and make predictions (specifically cells `KrcwlGEJscCH` and `Jy6EN5vfskUk`) for the changes to take effect.